### 1. 모듈

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [223]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd
import re
import lxml
from random import randint

### 2. url 접속

In [214]:
driver = webdriver.Chrome('/Users/harampark/Documents/chromedriver')

In [215]:
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [216]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

In [217]:
num_results = driver.find_element_by_id("pageSizeOption")
num_results.send_keys('100')

In [227]:
driver.close()

### 3. 크롤링

#### 본문 크롤링

In [218]:
# 총 282쪽까지 존재 
n = 15

for n in tqdm(range(1, n)): 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')  # table이란 태그를 찾아라! 
    
    df = pd.read_html(str(table))
    df_tmp = df[0]
    
    if n == 1: 
        bill_info_df = df_tmp
    else: 
        bill_info_df = pd.concat([bill_info_df, df_tmp])
    
    print(n)
    
    # 282쪽이면 다음 페이지로 넘어가지 않도록! 
    if n == 282: 
        break 
     
    # 페이지 넘김 
    if n % 10 != 0:
        if n < 10:
            paging = driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({(n%10)+1})')
        else: 
            paging = driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({(n%10)+3})')    
        paging.click()
        time.sleep(3)
        
    else:
        if n == 10: 
            next_paging = driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)')
        else: 
            next_paging = driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)')
        next_paging.click()
        time.sleep(3)
    
    MAX_SLEEP_TIME = 5
    rand_value = randint(3, MAX_SLEEP_TIME)
    time.sleep(rand_value)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [219]:
len(bill_info_df)

1400

In [220]:
bill_info_df.to_csv('bill-info-contents.csv', sep=',', encoding='utf-8')

#### 개별 의안 정보 크롤링

In [226]:
driver = webdriver.Chrome('/Users/harampark/Documents/chromedriver')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

num_results = driver.find_element_by_id("pageSizeOption")
num_results.send_keys('100')

In [224]:
# 총 282쪽까지 존재 
n = 2
contents = []
boards = []
members = []

for n in tqdm(range(1, n)):
    for bill_num in range(1,101):
        # 의안 누르기: tr 1-100까지 수정해주면 됨! 
        bill = driver.find_element_by_css_selector(f'tbody > tr:nth-child({bill_num}) > td.alignL > div.pl25 > a')
        bill.click()
        MAX_SLEEP_TIME = 5
        rand_value = randint(3, MAX_SLEEP_TIME)
        time.sleep(rand_value)
        
        # 1. 더보기 버튼 눌러주기 
        more_btn = driver.find_element_by_id('summaryContentMoreBtn')
        more_btn.click()
        MAX_SLEEP_TIME = 5
        rand_value = randint(3, MAX_SLEEP_TIME)
        time.sleep(rand_value)

        # 2. 해당 페이지 html 가져오기 
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table')

        # 제안날짜 
        date = table.find('tbody').find_all('td')[3].get_text()

        # 제안내용
        try:
            content_raw = soup.find('div', class_='textType02').get_text()    
            content_split = re.split('\r|\t|\n|\xa0', content_raw)
            content = ''.join(content_split)
            contents.append(content)
                
        except:
            contents.append('') 
    
        # 소관위원회
        try:
            board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
            boards.append(board)
            time.sleep(0.5)
        except:
            boards.append('')
    
        # 발의한 의원들 리스트 
        try: 
            proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
            proposer.click()
    
            # 새로운 창으로 초점 변경 
            driver.switch_to_window(driver.window_handles[1])
            driver.get_window_position(driver.window_handles[1])
    
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            members_raw = soup.find_all('div', class_='links textType02 mt20')[0]
            members = members_raw.find_all('a')
            members_list = []
            for n in range(len(members)):
                members_list.append(members[n].get_text())
            members.append(members_list)
    
            driver.close()
    
            # 원래 창으로 돌아가기
            driver.switch_to_window(driver.window_handles[0])
            driver.get_window_position(driver.window_handles[0])
    
        except: 
            members.append('')
    
        # 목록으로 돌아가기 
        list_btn = driver.find_element_by_css_selector('div.listBtn > a')
        list_btn.click()
        
        # 282쪽이면 다음 페이지로 넘어가지 않도록! 
    if n == 282: 
        break 
     
    # 페이지 넘김 
    if n % 10 != 0:
        if n < 10:
            paging = driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({(n%10)+1})')
        else: 
            paging = driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({(n%10)+3})')    
        paging.click()
        time.sleep(3)
        
    else:
        if n == 10: 
            next_paging = driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)')
        else: 
            next_paging = driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)')
        next_paging.click()
        time.sleep(3)
    
    MAX_SLEEP_TIME = 5
    rand_value = randint(3, MAX_SLEEP_TIME)
    time.sleep(rand_value)

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-224-eed39eea3aaa>:51: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
/usr/local/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
<ipython-input-224-eed39eea3aaa>:66: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])
  0%|          | 0/1 [00:30<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="summaryContentMoreBtn"]"}
  (Session info: chrome=84.0.4147.105)


In [80]:
# 의안 누르기: tr 1-100까지 수정해주면 됨! 
bill = driver.find_element_by_css_selector('tbody > tr:nth-child(42) > td.alignL > div.pl25 > a')
bill.click()
# time.sleep(1)

In [56]:
contents = []
boards = []
members = []

# 1. 더보기 버튼 눌러주기 
more_btn = driver.find_element_by_id('summaryContentMoreBtn')
more_btn.click()

# 2. 해당 페이지 html 가져오기 
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')

# 제안날짜 
date = table.find('tbody').find_all('td')[3].get_text()

# 제안내용
try:
    content_raw = soup.find('div', class_='textType02').get_text()    
    content_split = re.split('\r|\t|\n|\xa0', content_raw)
    content = ''.join(content_split)
    contents.append(content)
                
except:
    contents.append('') 
    
# 소관위원회
try:
    board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
    boards.append(board)
except:
    boards.append('')
    


In [81]:
# 발의한 의원들 리스트 
try: 
    proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
    proposer.click()
    
    # 새로운 창으로 초점 변경 
    driver.switch_to_window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    members_raw = soup.find_all('div', class_='links textType02 mt20')[0]
    members = members_raw.find_all('a')
    members_list = []
    for n in range(len(members)):
        members_list.append(members[n].get_text())
    members.append(members_list)
    
    driver.close()
    
    # 원래 창으로 돌아가기
    driver.switch_to_window(driver.window_handles[0])
    driver.get_window_position(driver.window_handles[0])
    
except: 
    members.append('')
    
# 목록으로 돌아가기 
list_btn = driver.find_element_by_css_selector('div.listBtn > a')
list_btn.click()